In [ ]:
import numpy as np
import xarray as xr
import holoviews as hv
from scipy.optimize import curve_fit
from scipy.constants import hbar, Boltzmann, k
hv.extension('bokeh')
#import setup
import os
#from transmission import Transmission_qcodes as tr
from qcodes import initialise_or_create_database_at, load_by_run_spec
#from fitting_tools import complex_fit
hv.extension("bokeh")
import skg #pip install scikit-guess to install this package
#import skgstat as skg


import pandas as pd


In [43]:
from qcodes import initialise_or_create_database_at, load_by_run_spec

In [44]:
from scipy.optimize import least_squares


def phase_Q(freqs, Qtot, theta0, f0):
    return theta0 + 2*np.arctan(2*Qtot*(1-freqs/f0))
    
def fit_renorm(S_renorm, freqs):
    x = np.real(S_renorm)
    y = np.imag(S_renorm)
    z= np.column_stack((x,y))

    r, c = skg.nsphere_fit(z)
    
    S_center = x-c[0] + 1j*(y-c[1])
    
    phi_0 = -np.arcsin(c[1]/r)
    
    def func_to_be_fitted_two(freqs, Qtot, f0):
        return phi_0+np.pi + 2*np.arctan(2*Qtot*(1-freqs/f0))
    
    phase = np.remainder(np.angle(S_center), 2*np.pi)
    amplitude = np.sqrt(x**2 +y**2).to_numpy()
#     f_init = freqs[np.argmin(phase)]
    f_init = freqs[np.argmin(amplitude)]
    f1 = freqs[np.argmax(np.abs(np.abs(0.5-amplitude/np.max(amplitude))-0.5))]
    #Q_init = f_init/(3*(np.abs(f_init-f1)))
    Q_init = 10000
    #print('Q_init = ', Q_init)
#     Q_init = 25000
    p0 = [Q_init, f_init]
    
    
    
    popt, pcov = curve_fit(func_to_be_fitted_two, freqs, phase, p0 = p0,ftol=1e-10, xtol=1e-10, gtol=1e-10)
    Ql = popt[0]
    fr = popt[1] 
    phase_plot = (
        
    hv.Scatter((freqs, phase), label = 'data')
    *hv.Curve((freqs, func_to_be_fitted_two(freqs, *popt)), label = 'fit').opts(color = 'black')
    *hv.Scatter((freqs, func_to_be_fitted_two(freqs, *p0)), label = 'guess').opts(color = 'green')
    ).opts(title = 'Phase fit after renormalisation')
    
    Qc = Ql/(2*r*np.exp(-1j*phi_0))
    Qi = 1/(1/Ql-np.real(1/Qc))
    
    return  phase_plot, Ql, fr, popt, pcov, Qc, Qi, phi_0, S_center, r,c 

def cable_delay_fitting(S, freqs):
    
    def func(tau):
        return S*np.exp(2*np.pi*1j*tau*freqs)

    
    def func_to_minimize(tau):
        S_tau = func(tau)
        x = np.real(S_tau).to_numpy()
        y = np.imag(S_tau).to_numpy()
        z= np.column_stack((x,y))

        r, c = skg.nsphere_fit(z)
        
        S_shift = S_tau-(c[0]+1j*c[1])
        return r-np.abs(S_shift)
    
    tau = least_squares(func_to_minimize, 30e-9 , method = 'trf', xtol = 1e-15, ftol = 1e-15 )
    
    return tau

def correct_S_by_delay(S, freqs, tau):
    return S*np.exp(2*np.pi*1j*tau*freqs)



In [45]:
def gen_circle(r, c, num=1001):
    angles = np.linspace(0, 2 * np.pi, num=num)
    xc = c[0] + r * np.cos(angles)
    yc = c[1] + r * np.sin(angles)
    return xc, yc

In [46]:
def normalize_transmission(S):
    I = np.real(S["S21"])
    Q = np.imag(S["S21"])
    S_numpy = S["S21"].to_numpy()
    freqs = S["pna_frequency_axis"]
    S = S["S21"]
    tau = cable_delay_fitting(S, freqs)
    # print("cable delay =", tau.x[0])
    # correction of S by cable delay to get a nice circular shape:
    St = correct_S_by_delay(S, freqs, tau.x[0])

    # fit to get radius and center of the circle:
    xt = np.real(St).to_numpy()
    yt = np.imag(St).to_numpy()
    zt = np.column_stack((xt, yt))
    rt, ct = skg.nsphere_fit(zt)

    x_circ, y_circ = gen_circle(c=ct, r=rt)

    #     plot_corrected_S = (
    #         hv.Scatter((np.real(St), np.imag(St)), label = f'corrected data by {np.round(tau.x[0]*1e9,3)} ns')
    #         *hv.Curve((x_circ,y_circ), label = 'circle fit')
    #         *hv.Scatter((np.real(P_tilde), np.imag(P_tilde)), label = 'off res. point').opts(line_width=2, line_color = 'black')
    #     ).opts(title = 'Transmission corrected by cable delay')

    # center the data to perform the phase fit:
    xx = xt - ct[0]
    yy = yt - ct[1]
    St_center = xx + 1j * yy
    # define the phase:
    phase = np.unwrap(np.angle(St_center))

    # initial params of the fit (to be added as optionnal arguments)
    f_init = freqs[np.argmin(phase)]
    #     print(f_init)
    Q_init = 10000  # TODO: change the initial value of Q
    theta_init = phase[0] - np.pi
    #     print('theta_ini)
    p0 = [Q_init, theta_init, f_init]

    # perform the fit:
    popt, pcov = curve_fit(phase_Q, freqs, phase, p0=p0)
    init_phase_curve = (
        hv.Scatter((freqs, phase), label="data")
        * hv.Curve((freqs, phase_Q(freqs, *popt)), label="fit").opts(line_color="black")
    ).opts(
        title="Initial phase fit (after delay removal)",
        ylabel="Phase (rad)",
        xlabel="Freq (Hz)",
    )
    #     Ql = popt[0]
    #     print("Ql=", Ql)
    #     Ql_err = pcov[0][0]
    #     fr = popt[2]
    #     fr_err = pcov[2][2]
    theta0 = popt[1]
    theta0_err = pcov[1][1]

    # definition of the P point that should be at (1,0)
    P_tilde = (
        ct[0] + rt * np.cos(theta0 + np.pi) + 1j * (ct[1] + rt * np.sin(theta0 + np.pi))
    )

    plot_corrected_S = (
        hv.Scatter(
            (np.real(St), np.imag(St)),
            label=f"corrected data by {np.round(tau.x[0]*1e9,3)} ns",
        )
        * hv.Curve((x_circ, y_circ), label="circle fit")
        * hv.Scatter((np.real(P_tilde), np.imag(P_tilde)), label="off res. point").opts(
            line_width=2, line_color="black"
        )
    ).opts(title="Transmission corrected by cable delay")
    # definition of attenuation amplitude and phase:
    alpha = np.angle(P_tilde)
    a = np.abs(P_tilde)

    S_renorm = 1 / a * np.exp(-1j * alpha) * St

    plot_S_renorm = hv.Scatter(
        (np.real(S_renorm), np.imag(S_renorm)), label="data"
    ).opts(title="renormalized transmission")

    # fit of the renormalized data:
    phase_plot, Ql, fr, popt, pcov, Qc, Qi, phi_0, S_center, r, c = fit_renorm(
        S_renorm, freqs
    )

    return (
        St,
        tau,
        rt,
        ct,
        plot_corrected_S,
        init_phase_curve,
        a,
        alpha,
        S_renorm,
        plot_S_renorm,
        phase_plot,
        Ql,
        fr,
        popt,
        pcov,
        Qc,
        Qi,
        phi_0,
        S_center,
        r,
        c,
        freqs,
    )


def ideal_transmissision(freqs, Ql, Qc, phi0, fr, a, alpha, tau):

    environment = a * np.exp(1j * alpha) * np.exp(-2 * np.pi * 1j * freqs * tau)
    cavity = 1 - (
        Ql / np.abs(Qc) * np.exp(1j * phi0) / (1 + 2 * 1j * Ql * (freqs / fr - 1))
    )

    return environment * cavity


def ideal_transmission_for_fit(freqs, Ql, Qc, phi0, fr, a, alpha, tau):
    N = len(freqs)
    freqs_real = freqs[: N // 2]
    freqs_imag = freqs[N // 2 :]

    transmission_real = np.real(
        ideal_transmissision(freqs_real, Ql, Qc, phi0, fr, a, alpha, tau)
    )
    transmission_imag = np.imag(
        ideal_transmissision(freqs_imag, Ql, Qc, phi0, fr, a, alpha, tau)
    )

    return np.hstack([transmission_real, transmission_imag])


def unwrap_complex_data(complex_data):
    return np.hstack([np.real(complex_data), np.imag(complex_data)])


def analyse_transmission(S, show_plots=False):
    (
        St,
        tau,
        rt,
        ct,
        plot_corrected_S,
        init_phase_curve,
        a,
        alpha,
        S_renorm,
        plot_S_renorm,
        phase_plot,
        Ql,
        fr,
        popt,
        pcov,
        Qc,
        Qi,
        phi_0,
        S_center,
        r,
        c,
        freqs,
    ) = normalize_transmission(S)

    reconstructed_S = ideal_transmissision(freqs, Ql, Qc, phi_0, fr, a, alpha, tau.x[0])

    result_dict = {
        "initial_data": S,
        "cable_delay_correction": {
            "corrected_data": St,
            "cable_delay": tau,
            "circle_fit": {
                "center": c,
                "radius": r,
            },
        },
        "attenuation": {
            "amplitude": a,
            "phase": alpha,
        },
        "renormalized_transmission": S_renorm,
        "Ql": Ql,
        "fr": fr,
        "phase_fit_results": {
            "popt": popt,
            "pcov": pcov,
        },
        "Qc": Qc,
        "Qi": Qi,
        "phi0": phi_0,
        "centered_normalized": {
            "centered_data": S_center,
            "circle_fit": {
                "center": c,
                "radius": r,
            },
        },
        "reconstructed_S": reconstructed_S,
    }

    if show_plots:
        plots = (
            (
                plot_corrected_S
                + init_phase_curve
                + plot_S_renorm
                + phase_plot
                + (
                    hv.Curve(
                        (np.real(reconstructed_S), np.imag(reconstructed_S)),
                        label="Reconstructed from fit",
                    ).opts(line_color="black")
                    * hv.Scatter((np.real(S["S21"]), np.imag(S["S21"])), label="data")
                )
            )
            .opts(title="Analysis summary", shared_axes=False)
            .cols(1)
        )

        return result_dict, plots
    return result_dict


def refined_fit(S, show_plots=False):
    result_dict = analyse_transmission(S)
    Ql_i = result_dict["Ql"]
    Qc_i = result_dict["Qc"]
    phi0_i = result_dict["phi0"]
    fr_i = result_dict["fr"]
    a_i = result_dict["attenuation"]["amplitude"]
    alpha_i = result_dict["attenuation"]["phase"]
    tau_i = result_dict["cable_delay_correction"]["cable_delay"].x[0]

    freqs = result_dict["initial_data"]["pna_frequency_axis"]
    #     freqs, Ql, Qc,phi_0, fr, a, alpha, tau.x[0])
    p0 = [
        Ql_i,
        np.abs(Qc_i),
        phi0_i,
        fr_i,
        a_i,
        alpha_i,
        tau_i,
    ]  # use fit results as initial parameters

    popt, pcov = curve_fit(
        ideal_transmission_for_fit,
        np.hstack([freqs, freqs]),
        unwrap_complex_data(S["S21"]),
        p0=p0,
    )

    #     reconstructed_S_init_fit = result_dict['reconstructed_S']
    reconstructed_S_init_fit = ideal_transmissision(freqs, *p0)
    reconstructed_S_refined_fit = ideal_transmissision(freqs, *popt)

    if show_plots:
        plots = (
            (
                hv.Curve(
                    (
                        np.real(reconstructed_S_init_fit),
                        np.imag(reconstructed_S_init_fit),
                    ),
                    label="initial fit",
                ).opts(line_color="black")
                * hv.Curve(
                    (
                        np.real(reconstructed_S_refined_fit),
                        np.imag(reconstructed_S_refined_fit),
                    ),
                    label="refined fit",
                ).opts(xlabel="Re(S21)", ylabel="Im(S21)", line_color="red")
                * hv.Scatter((np.real(S["S21"]), np.imag(S["S21"])), label="data")
                + (
                    hv.Curve(
                        (freqs / 1e9, np.abs(reconstructed_S_init_fit)),
                        label="init fit",
                    ).opts(line_color="black")
                    * hv.Curve(
                        (freqs / 1e9, np.abs(reconstructed_S_refined_fit)),
                        label="refined fit",
                    ).opts(
                        xlabel="Frequency (GHz)",
                        ylabel="Magnitude (ratio)",
                        line_color="red",
                    )
                    * hv.Scatter((freqs / 1e9, np.abs(S["S21"])), label="data")
                )
                + (
                    hv.Curve(
                        (freqs / 1e9, np.angle(reconstructed_S_init_fit)),
                        label="init fit",
                    ).opts(line_color="black")
                    * hv.Curve(
                        (freqs / 1e9, np.angle(reconstructed_S_refined_fit)),
                        label="refined fit",
                    ).opts(
                        xlabel="Frequency (GHz)", ylabel="Phase (deg)", line_color="red"
                    )
                    * hv.Scatter((freqs / 1e9, np.angle(S["S21"])), label="data")
                )
            )
            .opts(title="Analysis summary", shared_axes=False)
            .cols(3)
        )

        return popt, pcov, p0, plots

    return popt, pcov, p0

In [47]:
def S21(f,fc,Qi,Qc,phi,S0):
    delta_x = (f-fc)/fc
    a = 1+ Qi/Qc*np.exp(1j*phi)*1/(1+1j*2*Qi*delta_x)
    b=1/a
    T=20*np.log10(abs(b/S0))
    return T

def S21_env(f,a,alpha,tau,phi,fc,Ql,Qc):
    env=a*np.exp(1j*alpha)*np.exp(-2*np.pi*1j*f*tau)
    ideal_res=1-((Ql/np.abs(Qc)*np.exp(1j*phi))/(1+2*1j*Ql*(f/fc-1)))
    b=env*ideal_res
    T=20*np.log10(abs(b))
    return T
def _get_cov_fast_notch(
        self, xdata, ydata, fitparams
    ):    # enhanced by analytical derivatives
        # derivatives of notch_ideal model with respect to parameters
        def dS21_dQl(p, f):
            fr, absQc, Ql, phi0 = p
            return -(np.exp(1j * phi0) * fr**2) / (
                absQc * (fr + 2j * Ql * f - 2j * Ql * fr) ** 2
            )

        def dS21_dQc(p, f):
            fr, absQc, Ql, phi0 = p
            return (np.exp(1j * phi0) * Ql * fr) / (
                2j * (f - fr) * absQc**2 * Ql + absQc**2 * fr
            )

        def dS21_dphi0(p, f):
            fr, absQc, Ql, phi0 = p
            return -(1j * Ql * fr * np.exp(1j * phi0)) / (
                2j * (f - fr) * absQc * Ql + absQc * fr
            )

        def dS21_dfr(p, f):
            fr, absQc, Ql, phi0 = p
            return -(2j * Ql**2 * f * np.exp(1j * phi0)) / (
                absQc * (fr + 2j * Ql * f - 2j * Ql * fr) ** 2
            )

        u = self._residuals_notch_ideal_complex(fitparams, xdata, ydata)
        chi = np.absolute(u)
        u = u / chi  # unit vector pointing in the correct direction for the derivative

        aa = dS21_dfr(fitparams, xdata)
        bb = dS21_dQc(fitparams, xdata)
        cc = dS21_dQl(fitparams, xdata)
        dd = dS21_dphi0(fitparams, xdata)

        Jt = np.array(
            [
                aa.real * u.real + aa.imag * u.imag,
                bb.real * u.real + bb.imag * u.imag,
                cc.real * u.real + cc.imag * u.imag,
                dd.real * u.real + dd.imag * u.imag,
            ]
        )
        A = np.dot(Jt, np.transpose(Jt))
        chisqr = 1.0 / float(len(xdata) - len(fitparams)) * (chi**2).sum()
        try:
            cov = np.linalg.inv(A) * chisqr
        except:
            cov = None
        return chisqr, cov

In [ ]:
initialise_or_create_database_at(
    r"D:\Github\Qinu\qumin\data\2025-07-02_C12-036-J&C12-036-M.db"
)

In [ ]:
peak1 = load_by_run_spec(captured_run_id=7).to_xarray_dataset()
peak1["S21"] = peak1["pna_tr1_linear_magnitude"]*np.exp(1j*np.pi*peak1["pna_tr1_unwrapped_phase"]/180)
hv.Curve((peak1['pna_frequency_axis'],peak1['pna_tr1_magnitude']))


In [ ]:
peak1.to_dataframe().to_csv(r"D:\Github\Qinu\qumin\data\C12_30_H.csv",)


In [ ]:
%cd  D:\Github\Qinu\qumin\data
df = pd.read_csv("C12_30_H.csv")

a=df.index[df['pna_frequency_axis'] == 2e9][0]
a

In [ ]:
from scipy import signal
df = pd.read_csv(r"D:\Github\Qinu\qumin\data\C12_30_H.csv")

# Use np.isclose to find the nearest indices
idx_start = df.index[np.isclose(df['pna_frequency_axis'], 6e9)].min()
idx_end = df.index[np.isclose(df['pna_frequency_axis'], 7e9)].min()

if np.isnan(idx_start) or np.isnan(idx_end):
    raise ValueError("Could not find frequency values  in the data.")

peak2 = xr.Dataset.from_dataframe(df.iloc[idx_start:idx_end])

peak2["S21"] = peak2["pna_tr1_linear_magnitude"] * np.exp(
    1j * peak2["pna_tr1_unwrapped_phase"]
)

data = -1 * peak2["pna_tr1_magnitude"].to_numpy()
peakind = signal.find_peaks_cwt(data, 0.4e2)
hv.Curve((peak2["pna_frequency_axis"].values, peak2["pna_tr1_magnitude"].values))

In [ ]:
peak2
peakind
data
peak2["pna_frequency_axis"].values[peakind]

In [ ]:
# Use np.isclose to find the nearest indices
span =10e6
center_freq = peak2["pna_frequency_axis"].values[peakind[0]]
idx_start = df.index[np.isclose(df["pna_frequency_axis"], center_freq - span / 2)].min()
idx_end = df.index[np.isclose(df["pna_frequency_axis"], center_freq + span / 2)].min()


if np.isnan(idx_start) or np.isnan(idx_end):
    raise ValueError("Could not find frequency values close to 2e9 or 3e9 in the data.")

peak3 = xr.Dataset.from_dataframe(df.iloc[idx_start:idx_end])

hv.Curve((peak3["pna_frequency_axis"].values, peak2["pna_tr1_magnitude"].values)).opts(
    height=300,
    width=600,)

In [ ]:
import matplotlib.pyplot as plt
list_of_curves =[]
print(list_of_curves)
for i in range(len(peakind)):
    span = 1e4
    center_freq = peak2["pna_frequency_axis"].values[peakind[i]]
    idx_start = df.index[
        np.isclose(df["pna_frequency_axis"], center_freq - span / 2)
    ].min()
    idx_end = df.index[
        np.isclose(df["pna_frequency_axis"], center_freq + span / 2)
    ].min()

    # Use a new variable for the sliced dataset
    peak2_slice = xr.Dataset.from_dataframe(df.iloc[idx_start:idx_end])

    list_of_curves.append(
        hv.Curve(
            (
                peak2_slice["pna_frequency_axis"].values,
                peak2_slice["pna_tr1_magnitude"].values,
            )
        )
    )
    print(f"Peak {i+1} at {center_freq/1e9:.2f} GHz, index: {peakind[i]}")

# frequencies = [0.5, 0.75, 1.0, 1.25]


# def sine_curve(phase, freq):
#     xvals = [0.1 * i for i in range(100)]
#     return hv.Curve((xvals, [np.sin(phase + freq * x) for x in xvals]))


# curve_dict = {f: sine_curve(0, f) for f in frequencies}
# Create a dictionary mapping peak index to curve
# curve_dict = {f"Peak {i+1}": curve for i, curve in enumerate(list_of_curves)}
# hv.NdOverlay(curve_dict, kdims="peak")
# list_of_curves
from operator import mul
overlay = reduce(mul, list_of_curves)

In [ ]:
%cd  D:\Github\Qinu\qumin\data
df = pd.read_csv("C12_30_H.csv")
int1 = xr.Dataset.from_dataframe(df.iloc[(df.index[df['pna_frequency_axis'] == 2e9][0]): (df.index[df['pna_frequency_axis'] == 3e9][0])])

for var in int1.data_vars:
    data = int1[var]

    # If you want to create S21 for each variable, make sure the variable exists
    if "pna_tr1_linear_magnitude" in int1 and "pna_tr1_unwrapped_phase" in int1:
        S21 = int1["pna_tr1_linear_magnitude"] * np.exp(1j * int1["pna_tr1_unwrapped_phase"])
        # You can assign S21 to the dataset if needed:
        int1["S21"] = S21
        break  # Only need to do this once
    for var in int1.data_vars ['pna_tr1_linear_magnitude']:
        if var in int1.data_vars:
            int1[var] = int1[var].astype(float)
        


hv.Curve((int1['pna_frequency_axis'], int1['pna_tr1_magnitude']))
          


In [ ]:
def generate_start_stop_lists(start, stop, step):
    values = np.arange(start, stop + step, step)
    start_list = list(values[:-1])
    stop_list = [min(v + step, stop) for v in values[:-1]]
    return start_list, stop_list 

start_list, stop_list = generate_start_stop_lists(2, 9, 1)
start_list, stop_list


In [ ]:
freq=peak2['pna_frequency_axis']
fc_i=float((freq[0]+freq[-1])/2)
p0 = [3.24e9, 1e4, 1e3, -100, -8]  # f,fc,Qi,Qc,phi,S0 p0 = [fc_i, 1e4, 1e3, -100, -9]
popt, pcov = curve_fit(S21,freq,peak2['pna_tr1_magnitude'],p0)

Qi = np.format_float_scientific(np.abs(popt[1]),3)
Qc = np.format_float_scientific(np.abs(popt[2]),3)
Ql = np.format_float_scientific(1/(1/np.abs(popt[2])+1/np.abs(popt[1])),3)


(
    hv.Curve((freq,peak2['pna_tr1_magnitude']),label='data')*
    hv.Curve((freq,S21(freq, *popt)),label='fit')
).opts(legend_position='bottom_right',xlabel='Frequency (Hz)',ylabel='Magnitude (dB)',width=500,height=300,title='Qi = '+str(Qi)+' , Qc = '+str(Qc)+' & Ql = '+str(Ql))

In [57]:
initialise_or_create_database_at(
    r"D:\Github\Qinu\qumin\data\2025-08-29-C12-036-J-C12-037-AA.db"
)

In [58]:
import matplotlib.pyplot as plt
peak1 = load_by_run_spec(captured_run_id=31).to_xarray_dataset().sel({'pna_power':-30})
peak1["S21"] = peak1["pna_tr1_linear_magnitude"] * np.exp(
    1j * np.pi * peak1["pna_tr1_unwrapped_phase"] / 180
)
hv.Curve((peak1["pna_frequency_axis"], peak1["pna_tr1_magnitude"]))

:Curve   [x]   (y)

In [61]:
freq = peak1["pna_frequency_axis"]
fc_i = float((freq[0] + freq[-1]) / 2)
p0 = [fc_i, 1e5, 1e4, -100, -9]  # f,fc,Qi,Qc,phi,S0 p0 = [fc_i, 1e4, 1e3, -100, -9]
popt, pcov = curve_fit(S21, freq, peak1["pna_tr1_magnitude"], p0)

Qi = np.format_float_scientific(np.abs(popt[1]), 3)
Qc = np.format_float_scientific(np.abs(popt[2]), 3)
Ql = np.format_float_scientific(1 / (1 / np.abs(popt[2] ) + 1 / np.abs(popt[1])), 3)

curve=(
(
    hv.Curve((freq, peak1["pna_tr1_magnitude"]), label="data")
    * hv.Curve((freq, S21(freq, *popt)), label="fit")
).opts(
    legend_position="bottom_right",
    xlabel="Frequency (Hz)",
    ylabel="Magnitude (dB)",
    width=500,
    height=300,
    title="Qi = " + str(Qi) + " , Qc = " + str(Qc) + " & Ql = " + str(Ql),
))
curve

:Overlay
   .Curve.Data :Curve   [x]   (y)
   .Curve.Fit  :Curve   [x]   (y)

In [ ]:
hv.save(curve, r"D:\Github\rf-analysis-tools\notebook\fitting\C12fit.png")

In [ ]:
freq = peak1["pna_frequency_axis"]
fc_i = float((freq[0] + freq[-1]) / 2)
p0 = [fc_i, 1e5, 1e4, -100, -9]  # f,fc,Qi,Qc,phi,S0 p0 = [fc_i, 1e4, 1e3, -100, -9]
popt, pcov = curve_fit(S21, freq, peak1["pna_tr1_magnitude"], p0)

Qi = np.format_float_scientific(np.abs(popt[1]) , 3)
Qc = np.format_float_scientific(np.abs(popt[2]), 3)
Ql = np.format_float_scientific(
    1 / (1 / np.abs(popt[2] ) + 1 / np.abs(popt[1] )), 3
)

# curve=
(
    hv.Curve((freq, peak1["pna_tr1_magnitude"]), label="data")
    * hv.Curve((freq, S21(freq, *popt)), label="fit")
).opts(
    legend_position="bottom_right",
    xlabel="Frequency (Hz)",
    ylabel="Magnitude (dB)",
    width=500,
    height=300,
    title="Qi = " + str(Qi) + " , Qc = " + str(Qc) + " & Ql = " + str(Ql),
)
# hv.save(curve, r"D:\Github\Qinu\qumin\data\C12_I02_P_fit.png")

In [ ]:
I = np.real(peak1["S21"])
Q = np.imag(peak1["S21"])
hv.Scatter((I, Q)).opts(title="raw data")
hv.save(hv.Scatter((I, Q)).opts(title="raw data"), r"D:\Github\rf-analysis-tools\notebook\fitting\C12IQ.png")


In [ ]:
popt1, pcov1, initial_fit1, plots1 = refined_fit(peak1, show_plots=True)

fc = np.round(popt1[3] / 1e9, 3)
Qc = np.abs(np.real(popt1[1] * np.exp(-1j * popt1[2])))
Qi = np.abs(1 / (1 / popt1[0] - 1 / Qc))
Qc_str = np.format_float_scientific(Qc, 3)
Qi_str = np.format_float_scientific(Qi, 3)
Ql_str = np.format_float_scientific(popt1[0], 3)
Ql_str2 = 1 / (1 / float(Qc_str) + (1 / float(Qi_str)))

plots1.opts(
    title="Fitting parameters : fc = "
    + str(fc)
    + " GHz / Qc = "
    + str(Qc_str)
    + " / Qi = "
    + str(Qi_str)
    + " / Ql1 = "
    + str(Ql_str)
    + " / Ql2 = "
    + str(Ql_str2)
)
plots1

In [ ]:
freq=peak1['pna_frequency_axis']
fc_i=float((freq[0]+freq[-1])/2)
p0_env=[1,0.1,200e-9,0.1,fc_i,1e4,1e4] #a,alpha,tau,phi,fc,Ql,Qc
popt_env, pcov_env = curve_fit(S21_env,freq,peak1['pna_tr1_magnitude'],p0_env)

# Qi = np.format_float_scientific(popt_env[1],3)
Qc = np.format_float_scientific(-popt_env[-1],3)
Ql = np.format_float_scientific(-popt_env[-2],3)
Qi = np.format_float_scientific(-1/(1/popt_env[-2]-1/popt_env[-1]),3)


(
    hv.Curve((freq,peak1['pna_tr1_magnitude']),label='data')*
    hv.Curve((freq,S21_env(freq, *popt_env)),label='fit')
).opts(legend_position='bottom_right',xlabel='Frequency (Hz)',ylabel='Magnitude (dB)',width=500,height=300, title='Qi = '+str(Qi)+' , Qc = '+str(Qc)+' & Ql = '+str(Ql))

In [48]:
from qcodes import initialise_or_create_database_at, load_by_run_spec
initialise_or_create_database_at(
    r"D:\Github\Qinu\qumin\data\2025-08-29-C12-036-J-C12-037-AA.db"
)

In [49]:
Power = -20
peak = (load_by_run_spec(captured_run_id=28).to_xarray_dataset()).sel({'pna_power':Power})
peak["S21"] = peak["pna_tr1_linear_magnitude"]*np.exp(1j*np.pi*peak["pna_tr1_unwrapped_phase"]/180)
hv.Curve((peak['pna_frequency_axis'],peak['pna_tr1_magnitude']))
peak.to_dataframe().to_csv(r"D:\Github\Qinu\qumin\data\I.csv")

In [ ]:
Power = -30
peak = (load_by_run_spec(captured_run_id=88).to_xarray_dataset()).sel({'pna_power':Power})
freq=peak['pna_frequency_axis']
fc_i=float((freq[0]+freq[-1])/2)
p0=[fc_i,1e4,1e3,-80,-3]
popt, pcov = curve_fit(S21,freq,peak['pna_tr1_magnitude'],p0)

Qi = np.format_float_scientific(np.abs(popt[1]),3)
Qc = np.format_float_scientific(np.abs(popt[2]),3)
Ql = np.format_float_scientific(1/(1/np.abs(popt[2])+1/np.abs(popt[1])),3)


(
    hv.Curve((freq,peak['pna_tr1_magnitude']),label='data @P='+str(Power)+'dBm')*
    hv.Curve((freq,S21(freq, *popt)),label='fit')
).opts(legend_position='bottom_right',xlabel='Frequency (Hz)',ylabel='Magnitude (dB)',width=500,height=300,title='Qi = '+str(Qi)+' , Qc = '+str(Qc)+' & Ql = '+str(Ql))

In [50]:
power_min = -30
power_max = -10
power = []
fc = []
Ql = []
Qc = []
Qi = []
peak1 = (load_by_run_spec(captured_run_id=28).to_xarray_dataset()).sel({'pna_power':power_min})
curve=hv.Curve((peak1['pna_frequency_axis'],peak1['pna_tr1_magnitude']),label='data')
for p in range (power_min,power_max+1,2):
    peak = (load_by_run_spec(captured_run_id=28).to_xarray_dataset()).sel({'pna_power':p})
    freq=peak['pna_frequency_axis']
    fc_i=float((freq[0]+freq[-1])/2)
    p0=[fc_i,1e4,1e3,-80,-3]
    curve=curve * hv.Curve((freq,peak['pna_tr1_magnitude']),label='data')



In [51]:
curve

:Overlay
   .Curve.Data.I    :Curve   [x]   (y)
   .Curve.Data.II   :Curve   [x]   (y)
   .Curve.Data.III  :Curve   [x]   (y)
   .Curve.Data.IV   :Curve   [x]   (y)
   .Curve.Data.V    :Curve   [x]   (y)
   .Curve.Data.VI   :Curve   [x]   (y)
   .Curve.Data.VII  :Curve   [x]   (y)
   .Curve.Data.VIII :Curve   [x]   (y)
   .Curve.Data.IX   :Curve   [x]   (y)
   .Curve.Data.X    :Curve   [x]   (y)
   .Curve.Data.XI   :Curve   [x]   (y)
   .Curve.Data.XII  :Curve   [x]   (y)

In [ ]:
hv.save(curve, r"D:\Github\rf-analysis-tools\notebook\fitting\C12_non_lin.png")

In [ ]:
power_min = -20
power_max = -8
power = []
fc = []
Ql = []
Qc = []
Qi = []
for p in range(power_min, power_max + 1, 2):
    peak = (
        load_by_run_spec(captured_run_id=41)
        .to_xarray_dataset()
        .sel({'pna_power': p}, method='nearest')
    )
    freq = peak['pna_frequency_axis']
    fc_i = float((freq[0] + freq[-1]) / 2)
    p0 = [fc_i, 1e4, 1e3, -80, -3]
    popt, pcov = curve_fit(S21, freq, peak['pna_tr1_magnitude'], p0)
    Qi_i = np.abs(popt[1])
    Qc_i = np.abs(popt[2])
    Ql_i = 1 / (1 / np.abs(popt[2]) + 1 / np.abs(popt[1]))
    power.append(p)
    Qc.append(Qc_i)
    Qi.append(Qi_i)
    Ql.append(Ql_i)

In [ ]:
(
        hv.Curve((power,Qi),label='Qi').opts(logy=True)*
        hv.Curve((power,Qc),label='Qc').opts(logy=False)*
        hv.Curve((power,Ql),label='Ql').opts(logy=False)
).opts(xlabel='Power (dBm)',ylabel='Quality factor',legend_position='top_left',width=500,height=500)

In [ ]:
Ql_env=[3899.0,
 3783.0,
 3744.0,
 3880.0,
 3957.0,
 4029.0,
 4029.0,
 4080.0,
 4259.0,
 4237.0,
 4299.0,
 4330.0,
 4402.0,
 4436.0,
 4495.0,
 4518.0,
 4575.0,
 4609.0,
 4655.0,
 4675.0,
 4711.0,
 4774.0,
 4817.0,
 4721.0,
 4461.0,
 4745.0]

(
    hv.Curve((power,Ql_env),label='Fit with env').opts(logy=True)*
    hv.Curve((power,Ql),label='Fit without env').opts(logy=True)
).opts(xlabel='Power (dBm)',ylabel='Ql',legend_position='top_left',width=500,height=500)

In [ ]:
Delta=[]
for i in range(np.size(Qc)):
    Delta.append(100*np.abs(Ql[i]-Ql_env[i])/Ql_env[i])

(
    hv.Curve((power,Delta)).opts(xlabel='Power (dBm)',ylabel='Relative error (%)')
).opts(width=500,height=500)

In [ ]:
mode_index=range(5,15)#the resonance frequency is roughly 703MHz
resonance_frequency=[3.52462e9, 4.2275e9, 4.93084e9, 5.64036e9, 6.34555e9, 7.0475e9, 7.692e9, 8.488e9, 9.182e9, 9.875e9]
hv.Curve((mode_index,resonance_frequency))

In [ ]:
angular_frequency_index=[value *(np.pi)*2 for value in resonance_frequency]
angular_wavenumber_index=[value *(np.pi)/0.0075 for value in mode_index]#resonator_length=7.5mm
hv.Curve((angular_wavenumber_index,angular_frequency_index))

In [ ]:
def linear(x,a):
    y=a*x
    return y

popt, pcov = curve_fit(linear,angular_wavenumber_index,angular_frequency_index)
v_phi = popt[0]
Cl = 1.108e-10
Ll = 5.678e-07
Lkinl=1/(Cl*v_phi**2)-Ll
w=3e-6
Lkin_sq=Lkinl * w
print('Lkin_sq = ',Lkin_sq*1e12,' pH/sq')

In [ ]:
v_phi/1e7,np.sqrt(pcov[0][0])/1e7

In [ ]:
path=''
ds = xr.Dataset.from_dataframe(pd.read_csv(os.path.join(path, 'Simulation_test.csv'), header=0, sep='\t', index_col=['# Frequency (GHz)'] ,engine='python', skipfooter=1))


In [ ]:
hv.Curve((ds['# Frequency (GHz)'],ds['mag(S-parameters) (dB)']))

In [ ]:
freq=ds['# Frequency (GHz)']
fc_i=float((freq[0]+freq[-1])/2)
p0=[fc_i,1e4,1e3,-100,-9]
popt, pcov = curve_fit(S21,freq,ds['mag(S-parameters) (dB)'],p0)

Qi = np.format_float_scientific(np.abs(popt[1]),3)
Qc = np.format_float_scientific(np.abs(popt[2]),3)
Ql = np.format_float_scientific(1/(1/np.abs(popt[2])+1/np.abs(popt[1])),3)


(
    hv.Curve((freq,ds['mag(S-parameters) (dB)']),label='data')*
    hv.Curve((freq,S21(freq, *popt)),label='fit')
).opts(legend_position='bottom_right',xlabel='Frequency (Hz)',ylabel='Magnitude (dB)',width=500,height=300,title='Qi = '+str(Qi)+' , Qc = '+str(Qc)+' & Ql = '+str(Ql))